# Preprocess Imaging Data

For imaging data in this solution, we use the Computed Tomography (CT) series and the corresponding tumor segmentations in the NSCLC Radiogenomic imaging dataset to create patient-level 3-dimensional radiomic features that explain the size, shape and visual attributes of the tumors observed in the study subject’s lungs.

Medical imaging data is commonly stored in the [DICOM](https://www.dicomstandard.org/) file format, a standard that combines metadata and pixel data in a single object. For a volumetric scan, such as a lung CT scan, each cross-section slice is typically stored as an individual DICOM file. However, for ML purposes, analyzing 3-dimensional data provides a more wholistic view of the region of interest (ROI), thus providing better predictive values. We convert the scans in DICOM format into [NIfTI](https://nifti.nimh.nih.gov/) format. Thus, we download the DICOM files, store them to S3, then use using [Amazon SageMaker Processing](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html) to perform the transformation. Specifically, for each subject and study, we launch a SageMaker Processing job with a custom container to read the 2D DICOM slice files for both the CT scan and tumor segmentation, combine them to 3D volumes, save the volumes in NIfTI format, and write the NIfTI object back to S3. 

With the medical imaging data in volumetric format, we compute radiomic features describing the tumor region in the same SageMaker Processing job. Finally, the features engineered from each data modality are written to [Amazon SageMaker Feature Store](https://aws.amazon.com/sagemaker/feature-store/), a purpose-built repository for storing ML features. 

(optional) We run the medical imaging pipeline for each patient in a standalone SageMaker Processing job. This means we are distributing DICOM processing and radiomic feature computation on many SageMaker instances to speed up the process. 

## Step 1: Read in the SageMaker JumpStart Solution configuration

In [ ]:
import json

SOLUTION_CONFIG = json.load(open("stack_outputs.json"))
SOLUTION_BUCKET = SOLUTION_CONFIG["SolutionS3Bucket"]
REGION = SOLUTION_CONFIG["AWSRegion"]
SOLUTION_PREFIX = SOLUTION_CONFIG["SolutionPrefix"]
SOLUTION_NAME = SOLUTION_CONFIG["SolutionName"]
BUCKET = SOLUTION_CONFIG["S3Bucket"]
ECR_REPOSITORY = SOLUTION_CONFIG["SageMakerProcessingJobContainerName"]
CONTAINER_BUILD_PROJECT = SOLUTION_CONFIG["SageMakerProcessingJobContainerBuild"]
ACCOUNT_ID = SOLUTION_CONFIG["AccountID"]

Next step, we install [sagemaker-experiments](https://github.com/aws/sagemaker-experiments) package. We use [SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) to track and manage the SageMaker Processing jobs in this notebook. This allows us to view the status of the jobs for each patient in SageMaker Studio.

In [ ]:
!pip install -I sagemaker-experiments --no-index --find-links file://$PWD/wheelhouse

## Step 2: Understand the medical imaging pipeline
Working with large amount of DICOM files for a imaging study can be challenging. In this dataset, the source dataset is stored in DICOM format organized by patient/study/imaging sequence, as shown in the directory structure below.

```
R01-093/  # patient case ID
   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.240670240758603778272625719701/  # imaging study ID 1
   │   ├── 06-22-1994/  # date of imaging study 1
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.142853031813998654275885617348.json  # metadata for an imaging sequence (eg. CT or PET/CT)
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.142853031813998654275885617348/  # corresponding imaging sequence
   │   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.268122123877676186625117813999.dcm  # DICOM file(s) for the imaging sequence
   │   │   │   └── ... if there are more *.dcm files
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.143700250022821944637075337096.json
   │   └───┴── 1.3.6.1.4.1.14519.5.2.1.4334.1501.143700250022821944637075337096/
   │           └── *.dcm
   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.891373270862787721463778581588/  # imaging study ID 2
   │   ├── 08-31-1994/  # date of imaging study 2
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.127121510050458520043845333993/
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.127121510050458520043845333993.json
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.133664984480262659047625676012/
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.133664984480262659047625676012.json
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.172375466107449789684787272970/
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.172375466107449789684787272970.json
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.204884611948428816419269906683/
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.204884611948428816419269906683.json
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.257442018730697717273113074489/
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.257442018730697717273113074489.json
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.287138696590093616857690167236/
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.287138696590093616857690167236.json
   │   │   ├── 1.3.6.1.4.1.14519.5.2.1.4334.1501.327652266327905984847451930257/
   └───┴───┴── 1.3.6.1.4.1.14519.5.2.1.4334.1501.327652266327905984847451930257.json
```

In this study, we focus on the CT scans that are accompanied by a tumor segmentation, even though the PET and PET/CT scans are also available.
For each patient and study, we read through the json metadata files to determine if the series has both a CT scan imaging series and a segmentation object. If so, convert each CT scan imaging series, downloaded as a set of 2D DICOM files, to a single 3D NIfTI file. Then perform the DICOM to NIfTI conversion using a python package called [dcmstack](https://dcmstack.readthedocs.io/en/latest/), reading in all the DICOM files, sorting according to spatial slice location, and stacking the slices to create a 3D volume. The 3D volume is then written out in NIfTI format with the [NiBabel](https://nipy.org/nibabel/) python package. For each tumor segmentation DICOM object, use the [Pydicom](https://pydicom.github.io/) package to read in the 3D array, reorient the volume to match that of the corresponding CT scan, and save the output as a NIfTI file.

The medical imaging pipeline code is available in [./codebuild/build_container/dcm2nifti_processing.py](./codebuild/build_container/dcm2nifti_processing.py).

Note that the segmentation object for some studies were saved to a DICOM object with empty slices cropped out. This results in a mismatch between the number of slices in the CT scan and the corresponding segmentation object. To address this, match the value in the [ImagePositionPatient](https://dicom.innolitics.com/ciods/ct-image/image-plane/00200032) DICOM attribute to align the tumor segmentation to the corresponding location in the CT scan and pad the segmentation with zeros to have identical number of slices. 

The figure below shows example views of overlaying the tumor mask in yellow with transparency on the CT scan for a study (case ID R01-093).

![ct-R01-093-1](../docs/R01-093_06-22-1994_ortho-view.png)
![ct-R01-093-2](../docs/R01-093_06-22-1994_z-view.png)

Once the 3D volumes are constructed in NIfTI format, we compute the radiomic features describing the tumors in the annotated CT scans using the [pyradiomics](https://www.radiomics.io/pyradiomics.html) library. Using the library, we extract [120 radiomic features of 8 classes](https://pyradiomics.readthedocs.io/en/latest/features.html) such as statistical representations of the distribution and co-occurrence of the intensity within tumorous region of interest, and shape-based measurements describing the tumor morphologically. The computation of the radiomic features is performed volumetrically by providing the converted NIfTI images to the `RadiomicsFeatureExtractor` class in the `compute_features()` function in the [./codebuild/build_container/radiomics_utils.py](./codebuild/build_container/radiomics_utils.py) script.

```python
df = utils.compute_features(imageName, maskName)
```

The radiomic features computed from a pair of CT scan and segmentation are then ingested into a feature group in SageMaker Feature Store in [dcm2nifti_processing.py](./codebuild/build_container/dcm2nifti_processing.py).

```python 
feature_group = utils.create_feature_group(...)
feature_group.ingest(data_frame=df, ...)
```

## Step 3: Build a Container Image through Codebuild

To run the medical imaging pipeline using SageMaker Processing, we need to build a custom Docker container image from the [Dockerfile](./codebuild/build_container/Dockerfile). We are going to provisioned codebuild project to build an image and hold it in the Amazon ECR.

In [ ]:
import boto3
import run_codebuild

run_codebuild.build(CONTAINER_BUILD_PROJECT, boto3.session.Session(region_name=REGION))
ecr_image_uri = f"{ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/{ECR_REPOSITORY}:latest"
print(ecr_image_uri)

## Step 4: Set up SageMaker Experiment

We set up an [experiment](https://sagemaker-experiments.readthedocs.io/en/latest/experiment.html) using SageMaker Experiments to hold information of the medical imaging processing jobs. We can then easily track the jobs, and status in the **Experiments and trials** resources in the Studio UI.

In [ ]:
from botocore.config import Config

SM_BOTO = boto3.client('sagemaker', config=Config(connect_timeout=5, read_timeout=60, retries={'max_attempts': 20}), region_name=REGION)


In [ ]:
import time
from time import gmtime, strftime
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError

experiment_name = f'{SOLUTION_PREFIX}-nsclc'
trial_name = f'{SOLUTION_PREFIX}-trial'


# create the experiment if it doesn't exist
try:
    experiment = Experiment.load(experiment_name=experiment_name,
                                 sagemaker_boto_client=SM_BOTO)
except Exception as e:
    if "ResourceNotFound" in str(e):
        experiment = Experiment.create(
            experiment_name=experiment_name,
            sagemaker_boto_client=SM_BOTO,
            description='Lung cancer survival prediction using multi-modal Non Small Cell Lung Cancer (NSCLC) Radiogenomic dataset')
        print(f'{experiment_name} experiment is created!')


# create the trial if it doesn't exist
try:
    exp_trial = Trial.load(trial_name=trial_name,
                           sagemaker_boto_client=SM_BOTO)
except Exception as e:
    if "ResourceNotFound" in str(e):
        exp_trial = Trial.create(experiment_name=experiment_name, 
                                 sagemaker_boto_client=SM_BOTO,
                                 trial_name=trial_name)
        print(f'{exp_trial} trial is created!')

## Step 5: Create SageMaker Processing Job

We implement a function `launch_processing_job()` to launch a SageMaker processing job with the container image `ecr_image_uri` created in the previous step. 

In this solution, we run only the patients that have proper CT scans and tumor segmentations. There are a few imaging studies that come without segmentations or could not be constructed into NIfTI 3D volumes correctly in the original dataset, we remove those studies from the S3 bucket location to avoid spinning up unnecessary SageMaker Processing jobs. To gracefully skipping those studies, we put `script_processor.run()` in a try/except clause. 

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session(sagemaker_client=SM_BOTO)
dict_processor = {}

In [ ]:
import os
import time
from sagemaker.processing import ScriptProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput


def launch_processing_job(subject, input_data_s3, output_data_s3, feature_group_name, offline_store_s3uri, retries):
    
    exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
    jobname = f'{SOLUTION_PREFIX}-{subject}-{exp_datetime}' 

    experiment_config={'ExperimentName': experiment_name,
                       'TrialName': trial_name,
                       'TrialComponentDisplayName': f'ImageProcessing-{subject}'}

    inputs = [ProcessingInput(input_name='DICOM',
                              source=f'{input_data_s3}/{subject}', 
                              destination='/opt/ml/processing/input')]

    outputs = [ProcessingOutput(output_name=i,
                                source='/opt/ml/processing/output/%s' % i,
                                destination=os.path.join(output_data_s3, i)) 
               for i in ['CT-Nifti', 'CT-SEG', 'PNG']]

    arguments = ['--subject', subject, 
                 '--feature_group_name', feature_group_name, 
                 '--offline_store_s3uri', offline_store_s3uri]

    script_processor = ScriptProcessor(command=['python3'],
                                       image_uri=ecr_image_uri,
                                       role=sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.r5.large',
                                       volume_size_in_gb=5,
                                       sagemaker_session=sagemaker_session)
    current_retry = 1
    while True and current_retry <= retries:
        try:
            script_processor.run(code='codebuild/build_container/dcm2nifti_processing.py',
                                 inputs=inputs,
                                 outputs=outputs,
                                 arguments=arguments,
                                 job_name=jobname,
                                 experiment_config=experiment_config,
                                 wait=False,
                                 logs=False)
            return script_processor

        except ClientError as e:
            if "No S3 objects found under S3 URL" in str(e):
                print("Bad input data has been removed from s3, processing is not created!")
                return None
            elif "ResourceLimitExceeded" in str(e):
                if current_retry == retries:
                    raise
                else:
                    print("Resource reaches limit, please retry after 30 seconds ...")
                    current_retry += 1
                    time.sleep(30)
                    continue
            else:
                print(f'Processing job with {subject} subject is not created successfully! {e}.')
                raise

## Step 6: Run Processing Jobs for All Patients

To launch the SageMaker Processing jobs for multiple patients in parallel efficiently, we will run `launch_processing_job()` in a for loop, with each call submits a processing job to run on one `ml.r5.large` instance. In this case, we need to work with the service quota cleverly. 

The default service quota for *Number of instances across processing jobs* and *number of ml.r5.large instances* are 4 as documented in [Service Quota page](https://docs.aws.amazon.com/general/latest/gr/sagemaker.html#limits_sagemaker). If your account has a higher limit, you may run a higher number of simultaneous processing jobs (therefore faster completion time). To request a quota increase, please follow the [guidance](https://docs.aws.amazon.com/general/latest/gr/aws_service_limits.html). 

We implemented a function `wait_for_instance_quota()` to check for the current job count that is in `InProgress` state and limit the total count in this experiment to the value set in `job_limit`. If the total running job count is at the limit, the function waits number of seconds specified in `wait` argument and check the job count again. This is to account for account level SageMaker quota that may cause error in the for loop.

In [ ]:
def query_jobs(dict_processor):
    for key in list(dict_processor):
        if dict_processor[key]:
            status = dict_processor[key].jobs[-1].describe()['ProcessingJobStatus']
            if status in ["Completed", "Failed", "Stopped"]:
                del dict_processor[key]
        else:
            del dict_processor[key] # when no ProcessingJob created, i.e., None
    return len(dict_processor)


def wait_for_instance_quota(dict_processor, job_limit=4, wait=30):   
    job_count = query_jobs(dict_processor)    
    while job_count >= job_limit:
        print(f'Current total running jobs {job_count} is reaching the limit {job_limit}. Waiting {wait} seconds...')
        time.sleep(wait)
        job_count = query_jobs(dict_processor)
        
    print(f'Current total running jobs {job_count} is below {job_limit}. Proceeding...')
    return

Here, we set `job_limit` equals to 20 as an example, and it takes around 1 hour to complete all the processing jobs. You would expect a longer running time if you decrease the `job_limit`. 

We will be running this workflow for all the `RO1` subjects as defined in `subject_list` below.

In [ ]:
# this is where the DICOM files
input_data_bucket = f"{SOLUTION_BUCKET}-{REGION}" 
input_data_prefix = f"{SOLUTION_NAME}/data/nsclc_radiogenomics"

output_data_bucket=BUCKET
output_data_prefix="nsclc_radiogenomics"

input_dicom_dir = f"s3://{input_data_bucket}/{input_data_prefix}"
output_nifti_dir = f"s3://{output_data_bucket}/{output_data_prefix}"

imaging_feature_group_name = f'{SOLUTION_PREFIX}-imaging-feature-group'
%store imaging_feature_group_name

offline_store_s3uri = '%s/multimodal-imaging-featurestore' % output_nifti_dir

subject_list = ['R01-%03d'%i for i in range(1, 164)]

for subject in subject_list:
    print(subject)
    wait_for_instance_quota(dict_processor, job_limit=20, wait=30)
    dict_processor[subject] = launch_processing_job(subject, input_dicom_dir, output_nifti_dir, imaging_feature_group_name, offline_store_s3uri, 10)
    time.sleep(5)

Earlier we created an experiment to track our Processing Jobs. Let's take a look at how we can navigate to Processing Jobs from Experiments inside of SageMaker Studio.

<table>
    <tr>
        <td>
            <center>
                <img src="../docs/exp_1_icon.png" alt="SageMaker Resources Icon" style="width: 200px;"/>
                <p style="margin: 20px 80px 0px">To locate Experiments, first click on the SageMaker resources icon on the left side of the screen. The icon should be the last one on the left.</p>
            </center>
        </td>
        <td>
            <center>
                <img src="../docs/exp_2_drop_down.png" alt="Resource Drop Down" style="width: 200px;"/>
                <p style="margin: 20px 80px 0px">Inside of SageMaker resources, there is a drop-down underneath the SageMaker resources title and subject. Click the drop down. Here is where you can navigate to differernt SageMaker resources that are integrated with SageMaker Studio. Since we are looking for Experiments, select <b>Experiments and trials</b> from the drop down.</p>
            </center>
        </td>
    </tr>
    <tr>
        <td>
            <center>
                <img src="../docs/exp_3_select_experiment.png"       alt="Select an Experiment"           style="width: 200px;"/>
                <p style="margin: 20px 80px 0px">Inside of the Experiments and trials, we can see Experiments that were created. Let's navigate and double click on the first experiment titled <b>sagemaker-soln-lcsp-js-some random characters-nsclc</b> located under Name.</p>
            </center>
        </td>
        <td>
            <center>
                <img src="../docs/exp_4_select_trial.png"            alt="Select a Trial"                 style="width: 200px;"/>
                <p style="margin: 20px 80px 0px">Here is where we can find different trials within our Experiment. A trial consists of set of steps called trial components, and an example of a trial component is a Processing Job. Under Name, double click on the trial titled <b>sagemaker-soln-lcsp-js-some random characters-trial</b>.</p>
            </center>
        </td>
    </tr>
    <tr>
        <td>
            <center>
                <img src="../docs/exp_5_select_trial_components.png" alt="Select Trial Component"         style="width: 200px;"/>
                <p style="margin: 20px 80px 0px">This will take you to the Trial Components Section. Here is where we can find all of the Processing Jobs that were run for this trial. Double click on the first Processing Job titled <b>ImageProcessing-R01-some number</b> under Name.</p>
            </center>
        </td>
        <td>
            <center>
                <img src="../docs/exp_6_processing_job_metrics.png"         alt="Understanding a Processing Job" style="width: 400px;"/>
                <p style="margin: 20px 80px 0px">Within this Trial Component, you can select a number of items that were tracked for this Processing Job like: parameters, artifacts, and settings. </p>
            </center>    
        </td>
    </tr>
</table>

Programmatically, we can search for experiments, trials, and trial components using the SearchExpression method from the smexperiments library. Below shows us how to search and filter Processing jobs that are `InProgress` and the cell after shows an example of finding Processing jobs that may have `Failed`.

In [ ]:
from smexperiments.search_expression import Filter, Operator, SearchExpression
from smexperiments.trial_component import TrialComponent

search_expression_processing = SearchExpression(filters=[Filter(name='Status.PrimaryStatus',
                                                                operator=Operator.EQUALS,
                                                                value='InProgress'),
                                                         Filter(name='Parents.ExperimentName', 
                                                                operator=Operator.EQUALS, 
                                                                value=experiment_name),
                                                         Filter(name='Source.SourceType', 
                                                                operator=Operator.EQUALS, 
                                                                value='SageMakerProcessingJob')])
check = True

while check:
    num_processing_jobs = len(list(TrialComponent.search(search_expression=search_expression_processing)))
    if num_processing_jobs == 0:
        check = False
    else:
        print(f"{num_processing_jobs} processing jobs are still running. Check again after 30 sec ...") # add number of jobs
        time.sleep(30)
print("All processing jobs finished!")

In [ ]:
search_expression_failed = SearchExpression(filters=[Filter(name='Status.PrimaryStatus',
                                                                operator=Operator.EQUALS,
                                                                value='Failed'),
                                                         Filter(name='Parents.ExperimentName', 
                                                                operator=Operator.EQUALS, 
                                                                value=experiment_name),
                                                         Filter(name='Source.SourceType', 
                                                                operator=Operator.EQUALS, 
                                                                value='SageMakerProcessingJob')])


num_failed_jobs = len(list(TrialComponent.search(search_expression=search_expression_failed)))
print(f"{num_failed_jobs} jobs failed!")

Retrieve the experiment data for analysis and review.

In [ ]:
from sagemaker import analytics

trial_component_analytics = analytics.ExperimentAnalytics(experiment_name=experiment_name)

analytic_table = trial_component_analytics.dataframe()
analytic_table.head()

## Next Stage

Next, we'll move on to start the ML modeling.

Click here to [continue](./4_train_test_model.ipynb).